In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_dir = '/content/drive/My Drive/Colab Notebooks/soyo_project/Election/'

In [4]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
!pip install -q konlpy

     |████████████████████████████████| 19.4 MB 6.1 MB/s 
     |████████████████████████████████| 448 kB 70.9 MB/s 


In [35]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
def onlyhangul(x):
    pattern = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글 및 띄어쓰기에 해당되지 않는 부분(영어, 숫자, 기호, 이모티콘 등)
    results = pattern.sub('', str(x))
    # removed_string = pattern.findall(x)
    return results

def spellchecker(x):
    # Py-Hanspell 사용
    spelled_sent = spell_checker.check(x)
    hanspell_sent = spelled_sent.checked
    return hanspell_sent

In [41]:
from nltk.tokenize import word_tokenize

def remove_stopwords(x):
    result = []
    word_tokens = word_tokenize(str(x))
    for w in word_tokens:
        if w not in kor_stopwords:
            result.append(w)
    
    return ' '.join(result)

In [38]:
kor_stopwords = []
with open('/content/drive/My Drive/Colab Notebooks/졸논/korean_stopwords.txt', 'r') as f:
  for line in f:
    kor_stopwords.append(line.strip())

## AI Hub 데이터 로드

In [5]:
# 단발성
short_df = pd.read_excel(base_dir + '한국어_단발성_대화_데이터셋.xlsx')
short_df = short_df[['Sentence', 'Emotion']]
print(short_df.shape)
short_df.head()

(38594, 2)


,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [6]:
# 연속성
long_df = pd.read_excel(base_dir + '한국어_연속적_대화_데이터셋.xlsx')
long_df = long_df[['Unnamed: 1', 'Unnamed: 2']]
long_df = long_df.iloc[1:, :]
long_df.columns = ['발화', '감정']
print(long_df.shape)
long_df.head()

(55628, 2)


,발화,감정
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립


In [7]:
short_df['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [8]:
long_df = long_df.loc[long_df['감정'].isin(['중립', '놀람', '분노', '슬픔', '행복', '혐오', '공포'])]
long_df['감정'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: 감정, dtype: int64

In [9]:
long_df = long_df.rename(columns={'발화': 'Sentence', '감정': 'Emotion'})

In [27]:
pos_df = pd.concat([short_df.loc[short_df['Emotion'] == '행복'], long_df.loc[long_df['Emotion'] == '행복']])
neg_df = pd.concat([short_df.loc[short_df['Emotion'].isin(['분노', '혐오'])], long_df.loc[long_df['Emotion'].isin(['분노', '혐오'])]])
neu_df = pd.concat([short_df.loc[short_df['Emotion'] == '중립'], long_df.loc[long_df['Emotion'] == '중립']])

print(f'긍정: {pos_df.shape[0]}\n', f'부정: {neg_df.shape[0]}\n', f'중립: {neu_df.shape[0]}')

긍정: 7067
 부정: 14942
 중립: 48616


## NSMC 데이터 로드

In [21]:
nsmc_df = pd.read_table(base_dir + 'nsmc_ratings.txt', sep='\t')
print(nsmc_df.shape)
nsmc_df.head()

(200000, 3)


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [25]:
nsmc_df.drop(['id'], axis=1, inplace=True)
nsmc_df = nsmc_df.rename(columns={'document': 'Sentence', 'label': 'Emotion'})

In [29]:
# 중립 데이터와 비율 맞추기
pos_df = pd.concat([pos_df, nsmc_df.loc[nsmc_df['Emotion'] == 1][:40000]])
neg_df = pd.concat([neg_df, nsmc_df.loc[nsmc_df['Emotion'] == 0][:30000]])
print(pos_df.shape)
print(neg_df.shape)

(47067, 2)
(44942, 2)


In [54]:
# Multiclass -> 중립: 0, 긍정: 1, 부정: 2
neu_df['Emotion'] = 0
pos_df['Emotion'] = 1
neg_df['Emotion'] = 2
final_df = pd.concat([pos_df, neg_df, neu_df])
print(final_df.shape)
final_df.sample(5)

(140625, 2)


,Sentence,Emotion
111402,아~~~무생각없이보면....아 그래도 욕은 나오겠구나.... ㅡㅡ,2
109151,여배우들 섹시한 비주얼만 볼만하다,2
22527,잔잔한 물결이 큰 파도를 만드는 원동력이 될 수 있다. 10분의 3도 결코 작은 수...,0
50465,"바보야, 계속 진심이다. 서두르지만 않았지.",0
34006,2번도모르는국민이뽑은대통령이기에뭐라할말도엄네~~,2


In [55]:
# 한글만 + 불용어 제거
final_df['Cleaned'] = final_df['Sentence'].apply(lambda x: remove_stopwords(onlyhangul(x)))
final_df.sample(10)

,Sentence,Emotion,Cleaned
25340,선빈이와서10승이하투수들이랑만붙었잖아..이제15승급투수들이랑붙어보고..설레발치라,0,선빈이와서승이하투수들이랑만붙었잖아이제승급투수들이랑붙어보고설레발치라
15007,재밌게 보는 드라마 한개가 끝나니 너무 아쉽네요 ㅠ 정말 재미있었습니다. 시즌2 기...,1,재밌게 보는 드라마 한개가 끝나니 너무 아쉽네요 ㅠ 정말 재미있었습니다 시즌 기대 ...
12901,최순실을 사형해라!!!!!!!,2,최순실을 사형해라
26703,그럼 짜장면은?,0,짜장면은
8133,흐흐. 일할 게 있어서요.,0,일할 게 있어서요
120631,이거 보자는친구 그날 팼음..,2,이거 보자는친구 그날 팼음
25105,30~40대 아줌마들 진짜 댓글 많이 다네 ㅋㅋㅋㅋ,0,대 아줌마들 진짜 댓글 많이 다네 ㅋㅋㅋㅋ
103723,아무 생각없이 막 찍어대고 편집으로 이야기 만들어내는 감독,2,아무 생각없이 막 찍어대고 편집으로 이야기 만들어내는 감독
39899,감동을 느낄수 있어요.ㅋㅋ,1,감동을 느낄수 있어요ㅋㅋ
49782,견실한 조언，송구스럽습니다.,0,견실한 조언송구스럽습니다


## 전처리 데이터 저장

In [56]:
final_df.to_csv(base_dir + 'dataset/전처리.csv', index=False, encoding='utf-8-sig')